## Прогнозирование оттока клиентов банка

#### Описание проекта  
Исходные данные содержат информацию о клиентах банка. Основная задача - прогнозирование оттока клиентов.  

Задачи:

* Построить модель для прогнозирования оттока клиентов на основе данных о клиентах.
* Провести сегментацию клиентов по признакам (например, возраст, баланс, использование кредитных карт) для выделения типичных групп.
* Проанализировать признаки, оказывающие наибольшее влияние на отток клиентов.
* Сформулировать выводы и предложить рекомендации для уменьшения оттока и улучшения обслуживания клиентов.

Данные:  

customer_id: Уникальный идентификатор клиента.  
credit_score: Кредитный рейтинг клиента.  
country: Страна проживания клиента.  
gender: Пол клиента.  
age: Возраст клиента.  
tenure: Количество лет, которое клиент провел в банке.  
balance: Баланс на счете клиента.  
products_number: Количество продуктов, которые клиент использует в банке.  
credit_card: Наличие кредитной карты (0 — нет, 1 — есть).  
active_member: Статус активного клиента (0 — неактивный, 1 — активный).  
estimated_salary: Примерный доход клиента.  
churn: Целевой признак, указывающий на отток клиента (1 — клиент ушел, 0 — остался).  

#### Оглавление  
- [1. Загрузка данных](#1-Загрузка-данных)  
- [2. Анализ данных](#2-Анализ-данных)  
- [3. Построение модели](#3-Построение-модели)  
- [4. Выводы](#4-Выводы)

#### 1. Загрузка данных <a id="1-Загрузка-данных"></a>  

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

# Загружаем данные
import os
data_path = os.path.join(os.getcwd(), 'data', 'bank_customer_churn.csv')
data = pd.read_csv(data_path)

# Просмотр первых строк данных
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       10000 non-null  int64  
 1   credit_score      10000 non-null  int64  
 2   country           10000 non-null  object 
 3   gender            10000 non-null  object 
 4   age               10000 non-null  int64  
 5   tenure            10000 non-null  int64  
 6   balance           10000 non-null  float64
 7   products_number   10000 non-null  int64  
 8   credit_card       10000 non-null  int64  
 9   active_member     10000 non-null  int64  
 10  estimated_salary  10000 non-null  float64
 11  churn             10000 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 937.6+ KB


In [3]:
data.describe()

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
profile = ProfileReport(data, minimal=True)

profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# Расчет средних значений признаков для клиентов, которые ушли (1) и остались (0)
numeric_data = data.select_dtypes(include=['number'])
numeric_data.groupby(by='churn').mean() 

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
churn,,,,,,,,,
0,1.569117e+07,651.853196,37.408389,5.033279,72745.296779,1.544267,0.707146,0.554565,99738.391772
1,1.569005e+07,645.351497,44.837997,4.932744,91108.539337,1.475209,0.699067,0.360825,101465.677531
